# Artificial Intelligence Nanodegree
## Introduction to Game Playing
----

## Minmax values

![minimax_1.png](images/minimax_1.png)

서로가 번갈아 가면서 두는 게임이기 때문에 최상단 레이어가 값이 Max가 되는 행동을 하면, 다음 레이어에서 상대방은 Min이 되는 행동을 하게 된다.

## Game State Representation

In [1]:
from copy import deepcopy

xlim, ylim = 3, 2  # board dimensions

class GameState:
    """
    Attributes
    ----------
    _board: list(list)
        Represent the board with a 2d array _board[x][y]
        where open spaces are 0 and closed spaces are 1
    
    _parity: bool
        Keep track of active player initiative (which
        player has control to move) where 0 indicates that
        player one has initiative and 1 indicates player 2
    
    _player_locations: list(tuple)
        Keep track of the current location of each player
        on the board where position is encoded by the
        board indices of their last move, e.g., [(0, 0), (1, 0)]
        means player 1 is at (0, 0) and player 2 is at (1, 0)
    
    """

    def __init__(self):
        self._board = [[0] * ylim for _ in range(xlim)] #Numpy 아닌 일반 Python 배열에서는 곱셉하면 그 수 만큼 같은 수로 들어간다.
        self._board[-1][-1] = 1 # block lower-right corner #우 하단 막기.
        self._parity = 0 #순서. 처음 시작이므로 플레이어 0의 차례.
        self._player_locations = [None, None] #각 플레이어의 현재 위치. 처음 시작이므로 아직 None.
    
    def forecast_move(self, move):
        """ 
        Return a new board object with the specified move
        applied to the current game state.
        
        Parameters
        ----------
        move: tuple
            The target position for the active player's next move
        """
        #move를 적용해 현재 게임 상태를 반환한다.
        if move not in self.get_legal_moves(): #움직일 수 있는 리스트에 move가 없으면 에러.
            raise RuntimeError("Attempted forecast of illegal move")
            
        newBoard = deepcopy(self) #deepcopy를 사용하면 다른 메모리를 가지기 때문에 요소가 바뀌어도 원본에 영향을 끼치지 않는다.
        #일반적으로 메모리 사용량을 줄이기 위해 bitBoard를 사용한다.
        newBoard._board[move[0]][move[1]] = 1 #주어진 move 좌표의 값을 1로 바꿔 사용된 것으로 한다.
        newBoard._player_locations[self._parity] = move #현재 플레이어의 위치를 move로 바뀌준다.
        newBoard._parity ^= 1 #xor. 0이면 1, 1이면 0이 된다. 다음 플레이어 턴으로 설정.
        
        return newBoard
    
    def get_legal_moves(self):
        """ 
        Return a list of all legal moves available to the
        active player.  Each player should get a list of all
        empty spaces on the board on their first move, and
        otherwise they should get a list of all open spaces
        in a straight line along any row, column or diagonal
        from their current position. (Players CANNOT move
        through obstacles or blocked squares.) Moves should
        be a pair of integers in (column, row) order specifying
        the zero-indexed coordinates on the board.
        """
        #움직일 수 있는 모든 목록 반환
        loc = self._player_locations[self._parity] #현재 플레이어의 위치를 가져온다.
        if not loc: #현재 플레이어 위치가 없다면 (처음의 경우)
            return self._get_blank_spaces() #빈 공간을 반환(2번째 플레이어라도 첫 플레이어가 둔 첫 수 외 모든 위치가 가능하기 때문에)
        
        moves = []
        rays = [(1, 0), (1, -1), (0, -1), (-1, -1),
                (-1, 0), (-1, 1), (0, 1), (1, 1)] #한 칸 이동할 수 있는 경우의 수. →↘↓↙ ←↖↑↗
        
        for dx, dy in rays:
            _x, _y = loc
            while 0 <= _x + dx < xlim and 0 <= _y + dy < ylim: #현재 위치에서 주어진 ray 대로 이동할 수 있는 경우 (limit 이내)
                _x, _y = _x + dx, _y + dy #ray대로 한 칸 이동
                if self._board[_x][_y]: #이동한 위치가 1이라면(이미 사용됐던 위치라면) 
                    break #종료 후 다음 ray로 넘어간다.(for)
                moves.append((_x, _y)) #이동한 위치가 0이라면(사용 가능하다면), moves에 추가하고 같은 방향으로 한 칸 더 이동 가능한지 점검(while)
        
        return moves
    
    def _get_blank_spaces(self):
        """ Return a list of blank spaces on the board."""
        #빈 공간을 반환한다.
        return [(x, y) for y in range(ylim) for x in range(xlim)
                if self._board[x][y] == 0] #0으로 되어 있는 경우. 빈 공간

In [2]:
print("Creating empty game board...")
g = GameState()

print("Getting legal moves for player 1...")
p1_empty_moves = g.get_legal_moves()
print("Found {} legal moves.".format(len(p1_empty_moves or [])))

print("Applying move (0, 0) for player 1...")
g1 = g.forecast_move((0, 0))

print("Getting legal moves for player 2...")
p2_empty_moves = g1.get_legal_moves()
if (0, 0) in set(p2_empty_moves):
    print("Failed\n  Uh oh! (0, 0) was not blocked properly when " +
          "player 1 moved there.")
else:
    print("Everything looks good!")

Creating empty game board...
Getting legal moves for player 1...
Found 5 legal moves.
Applying move (0, 0) for player 1...
Getting legal moves for player 2...
Everything looks good!


## Implementing the Minimax Algorithm
Assumption 1: a state is terminal if the active player has no remaining moves     
Assumption 2: the board utility is -1 if it terminates at a max level, and +1 if it terminates at a min level

In [3]:
#1. 움직일 수 있는 나머지 공간이 없다면 종료
#2. Max레벨에서 종료되면 -1, Min레벨에서 종료되면 1

def terminal_test(gameState):
    """ 
    Return True if the game is over for the active player
    and False otherwise.
    """
    return not bool(gameState.get_legal_moves()) # by Assumption 1
    #게임이 더 진행 될 수 있으면 False, 아니면 True.

def min_value(gameState): #상대방의 턴. 상대방은 결과가 최소(상대방 승리)가 되는 값(-1)을 찾아야 한다.
    """ 
    Return the value for a win (+1) if the game is over,
    otherwise return the minimum value over all legal child
    nodes.
    """
    if terminal_test(gameState): #게임이 종료되는 조건. min_value를 찾아야 하는 레이어(상대방의 턴)에서 게임이 종료되면 1
        return 1 # by Assumption 2
    
    v = float("inf") #무한. 어떤 값보다 크다. #import math test = math.inf
    for m in gameState.get_legal_moves(): #움직일 수 있는 나머지 모든 위치를 가져온다.
        v = min(v, max_value(gameState.forecast_move(m))) #각 위치의 Max값을 가져와 비교. 최소값이 남게 된다.
        #다음 레이어(자신의 턴)에서 최대값. 이번 레이어(상대방 턴)에서 최소값.
        
    return v

def max_value(gameState): #자신의 턴. 자신은 결과가 최대(자신 승리)가 되는 값(1)을 찾아야 한다.
    """ 
    Return the value for a loss (-1) if the game is over,
    otherwise return the maximum value over all legal child
    nodes.
    """
    if terminal_test(gameState): #게임이 종료되는 조건. max_value를 찾아야 하는 레이어(자신의 턴)에서 게임이 종료되면 -1
        return -1 # by Assumption 2
    
    v = float("-inf") #-무한. 어떤 값보다 작다.
    for m in gameState.get_legal_moves(): #움직일 수 있는 나머지 위치를 가져온다.
        v = max(v, min_value(gameState.forecast_move(m))) #각 위치의 Min값을 가져와 비교. 최대값이 남게 된다.
        #다음 레이어(상대방 턴)에서 최소값. 이번 레이어(자신의 턴)에서 최대값.
        
    return v

##### 위의 그림 참조 해서 볼 것. Min 레이어 - Max 레이어.

In [4]:
g = GameState()

print("Calling min_value on an empty board...")
v = min_value(g)

if v == -1:
    print("min_value() returned the expected score!") #빈 보드라면 min_value == -1, max_value == 1이 되어야 한다.
else:
    print("Uh oh! min_value() did not return the expected score.")

Calling min_value on an empty board...
min_value() returned the expected score!


## Implement

In [5]:
def minimax_decision(gameState):
    """ 
    Return the move along a branch of the game tree that
    has the best possible value.  A move is a pair of coordinates
    in (column, row) order corresponding to a legal move for
    the searching player.
    
    You can ignore the special case of calling this function
    from a terminal state.
    """
    best_score = float("-inf") 
    best_move = None
    
    for m in gameState.get_legal_moves(): #이동할 수 있는 모든 곳 중에서
        v = min_value(gameState.forecast_move(m)) #max가 되는 값을 찾아야 하므로 다음 레이어의 min_value를 찾아야 한다.
        if v > best_score:
            best_score = v
            best_move = m
            
    return best_move

In [6]:
best_moves = set([(0, 0), (2, 0), (0, 1)])
rootNode = GameState()
minimax_move = minimax_decision(rootNode)

print("Best move choices: {}".format(list(best_moves)))
print("Your code chose: {}".format(minimax_move))

if minimax_move in best_moves:
    print("That's one of the best move choices. Looks like your minimax-decision function worked!")
else:
    print("Uh oh...looks like there may be a problem.")

Best move choices: [(0, 1), (2, 0), (0, 0)]
Your code chose: (0, 0)
That's one of the best move choices. Looks like your minimax-decision function worked!


In [11]:
#같은 값을 반환하지만, lambda와 max() 활용

# This solution does the same thing using the built-in `max` function
# Note that "lambda" expressions are Python's version of anonymous functions
def minimax_decision(gameState):
    """ 
    Return the move along a branch of the game tree that
    has the best possible value.  A move is a pair of coordinates
    in (column, row) order corresponding to a legal move for
    the searching player.
    
    You can ignore the special case of calling this function
    from a terminal state.
    """
    # The built in `max()` function can be used as argmax!
    return max(gameState.get_legal_moves(),
               key=lambda m: min_value(gameState.forecast_move(m)))